# Bayesian Combination Model

Example notebook for the BrainGPT project

## Load libraries

In [ ]:
import os
import logging
import submitit

import numpy as np
import pandas as pd

import torch

from haico.Bayesian_HM_model import BayesianCombinationModel

## Logger setup

In [ ]:
def setup_logging(logsdir="logs"):
    # get the hostname
    hostname = os.uname().nodename
    
    if 'SUBMITIT_EXECUTOR' in os.environ:
        logger = logging.getLogger("submitit") # using submitit task logger
        print(f'using submitit logger at {hostname}')
    else :
        # using hostname as the logger name
        logger = logging.getLogger(hostname)
        logger.setLevel(logging.DEBUG)
    
    # avoid duplicated handlers (duplicated log messages)
    if logger.hasHandlers():
        return logger

    if not os.path.exists(logsdir):
        os.makedirs(logsdir)

    # today date
    import datetime

    today = datetime.datetime.now().strftime("%Y_%m_%d")
    logfile = os.path.join(logsdir, "output_{}.log".format(today))
    formatter = logging.Formatter("{levelname} [{name}]: {asctime} - {message}", style="{")

    # Console handler
    chandler = logging.StreamHandler()
    chandler.setLevel(logging.DEBUG)
    chandler.setFormatter(formatter)
    logger.addHandler(chandler)

    # File handler
    fhandler = logging.FileHandler(logfile, "a")
    fhandler.setLevel(logging.DEBUG)
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    logger.info(f"Logging to: {logfile}")

    return logger

# get the parent directory of the current directory
parentdir = os.path.dirname(os.getcwd())

# using logger to log messages on the console and in a file
logsdir = os.path.join(parentdir, "logs")
logger = setup_logging(logsdir)

## Define parameters

In [ ]:
# Set a random seed for PyTorch
seed = 0
torch.manual_seed(seed)

# If using CUDA, set the random seed for CUDA
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    
# Set parameters for the Bayesian model
num_samples, warmup_steps, num_chains = 25, 500, 3

## Prepare human-machine data

In [ ]:
# Set path to the data directory
root_path = '../data/'

# List of classifiers to be analyzed
selected_LLMs = ['meta-llama--Llama-2-7b-chat-hf', 
                 'meta-llama--Llama-2-13b-chat-hf', 
                 'meta-llama--Llama-2-70b-chat-hf']

In [ ]:
# Read human participants data
online_study = pd.read_csv(f"{root_path}human/data/participant_data.csv")

# Select GPT-4 generated abstracts
abstract_idx = online_study['journal_section'].str.startswith('machine')
online_study = online_study[abstract_idx]

# Extract DOI links from all test cases assessed by human participants
doi = pd.read_csv(f"{root_path}human/abstract_id_doi.csv")
doi = doi['DOI;abstract_id;abstract'].str.split(';', expand=True)[[0,1]]
doi.columns = ['doi', 'abstract_id']

# Extract DOI links from GPT-4 generated test cases
gpt4_doi = pd.read_csv(f"{root_path}testcases/BrainBench_GPT-4_v0.1.csv")

# Reorder human participants data based on the order of GPT-4 generated abstracts
gpt4_order = gpt4_doi.merge(doi, on='doi')['abstract_id'].astype(float)
online_study['abstract_id'] = pd.Categorical(online_study['abstract_id'], categories=gpt4_order, ordered=True)
online_study = online_study.sort_values('abstract_id')

In [ ]:
# Initialize classification and confidence dataframes
classification = pd.DataFrame()
classification.loc[:,'abstract_id'] = np.array([np.where(gpt4_order==i)[0][0] for i in online_study['abstract_id']])
confidence = classification.copy()

# Set ground truth labels
order_labels = np.load(f"{root_path}machine/model_results/{selected_LLMs[0]}/llm_abstracts/labels.npy")
classification = classification.merge(pd.DataFrame(order_labels, columns=['true labels']), left_on='abstract_id', right_index=True)

# Set human classification and confidence
classification.loc[:,'Human'] = np.array([j if i == 1 else 1 - j for i, j in zip(online_study['correct'], classification['true labels'])])
confidence.loc[:,'Human'] = np.where(online_study['confidence'].values > 66, 2, np.where(online_study['confidence'].values <= 33, 0, 1)) 

In [ ]:
def softmax(x, axis=None):
    e_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

In [ ]:
for i in selected_LLMs:
    
    # Read PPL scores of machine classifiers
    machine_PPL = np.load(f"{root_path}machine/model_results/{i}/llm_abstracts/PPL_A_and_B.npy")
    
    # Get classification results
    machine_name = i.lstrip('meta-llama--Llama-2-').rstrip('-chat-hf').upper()
    machine_classification = pd.DataFrame(np.argmin(machine_PPL, axis=1), columns=[machine_name])
    classification = classification.merge(machine_classification, left_on='abstract_id', right_index=True)

    # Define confidence as PPL difference
    machine_confidence = pd.DataFrame(softmax(machine_PPL, axis=1), columns=[machine_name+'-A',machine_name+'-B'])
    confidence = confidence.merge(machine_confidence, left_on='abstract_id', right_index=True)

## Run Bayesian combination model

In [ ]:
# Define process cross_validation_fold as a function
def process_cross_validation_fold(fold, machine_clf, N):
    
    # Train data for current fold
    machine_probscores_train = torch.from_numpy(confidence[confidence['abstract_id']!=fold][[machine_clf+'-A',machine_clf+'-B']].values)
    human_classification_train = torch.from_numpy(classification[classification['abstract_id']!=fold]['Human'].values).to(torch.int64)
    human_confidence_train = torch.from_numpy(confidence[confidence['abstract_id']!=fold]['Human'].values)
    truelabel_train = torch.from_numpy(classification[classification['abstract_id']!=fold]['true labels'].values).to(torch.int64)
    
    # Test data for current fold
    machine_probscores_test = torch.from_numpy(confidence[confidence['abstract_id']==fold][[machine_clf+'-A',machine_clf+'-B']].values)
    human_classification_test = torch.from_numpy(classification[classification['abstract_id']==fold]['Human'].values).to(torch.int64)
    human_confidence_test = torch.from_numpy(confidence[confidence['abstract_id']==fold]['Human'].values)
    classification_test = classification[classification['abstract_id']==fold]
    
    # Initialize model
    model = BayesianCombinationModel()
    
    # This is the training phase (labels are observed)
    logger.info(f"Fold {str(fold+1).rjust(3, ' ')}/{N} | Training...")
    train_trace = model.infer(machine_probscores_train,
                              human_classification_train,
                              human_confidence_train,
                              truelabel=truelabel_train,
                              num_samples=num_samples,
                              warmup_steps=warmup_steps,
                              num_chains=num_chains,
                              disable_progbar=True,
                              group_by_chain=False)

    # Get model parameters for prediction
    params = model.params
    for key in params.keys():
        try:
            # Get the mean of the posterior samples, sort is for the case of cutpoints
            trace_now = train_trace[key].view(num_chains*num_samples,-1)
            trace_now = torch.sort(trace_now, dim=-1)[0].detach()
            params[key] = torch.mean(trace_now, dim=0)
        except:
            continue

    # This is the testing phase (labels are latent)
    logger.info(f"Fold {str(fold+1).rjust(3, ' ')}/{N} | Testing...")
    test_trace = model.infer(machine_probscores_test,
                             human_classification_test,
                             human_confidence_test,
                             truelabel=None, params=params,
                             num_samples=num_samples,
                             warmup_steps=warmup_steps, 
                             num_chains=num_chains,
                             disable_progbar=True,
                             group_by_chain=False)
    
    # Get predictions for this fold
    A_pred = classification_test[machine_clf].values == classification_test['true labels'].values
    B_pred = classification_test['Human'].values == classification_test['true labels'].values
    AB_all = torch.argmax(test_trace['labelprob'], dim=-1)
    AB_pred = torch.mode(AB_all, dim=0)[0].numpy() == classification_test['true labels'].values
    
    # Display predictions
    logger.info(f"Fold {str(j+1).rjust(3, ' ')}/{N} | A: {A_pred.mean():.3f} | B: {B_pred.mean():.3f} | AB: {AB_pred.mean():.3f}")

    # Save prediction accuracy
    predictions = pd.DataFrame({'A': A_pred, 'B': B_pred, 'AB': AB_pred})
    
    # Return the results
    return predictions

In [ ]:
N = len(gpt4_doi)

logger.debug(f"Number of abstracts: {N}")
logger.debug(f"selected_LLMs: {selected_LLMs}")

logs_submitit = os.path.join(logsdir, 'submitit')
logger.info(f'Logs dir submitit: {logs_submitit}')
executor = submitit.SlurmExecutor(folder=logs_submitit)

executor.update_parameters(
    partition="CPU,GPU",
    time="6:00:00", # 4 hours
    mem="350G",
    cpus_per_task=48,
    comment="haico-job",
    job_name="haico-job",
    array_parallelism=100,
)

for i in range(len(selected_LLMs)):
    
    machine_clf = classification.columns[3+i]
    logger.info(f"Classifier A: {machine_clf} --- Classifier B: Human")

    # Create a list to store the jobs
    jobs = []

    # Using array jobs with 100 parallel jobs
    with executor.batch():
        # Submit the job for each fold in parallel
        for j in range(N):
            job = executor.submit(process_cross_validation_fold, j, machine_clf, N)
            jobs.append(job)

    # Wait for all jobs to complete
    results = [job.result() for job in jobs]

    # Read the results into lists
    predictions_list = [result for result in results]

    # Convert lists to pandas dataframes
    predictions = pd.concat(predictions_list)

    # Save predictions for current model
    predictions.to_csv(f"../results/Bayesian_HM_predictions_{machine_clf}.csv", index=False)